# Rasteret example notebook that gives Xarray outputs



In [1]:
import nest_asyncio
from pathlib import Path
from shapely.geometry import Polygon
from rasteret import Rasteret
from rasteret.constants import DataSources
import xarray as xr

# Import necessary libraries and apply nest_asyncio
nest_asyncio.apply()


In [ ]:

workspace_dir = Path.home() / "rasteret_workspace"
workspace_dir.mkdir(exist_ok=True)

# Define a custom name for the stac index
custom_name = "bangalore_sentinel2_nb"

# Define area, time range and data source required in stac search
date_range = ("2025-01-01", "2025-01-30")
data_source = DataSources.SENTINEL2  # or SENTINEL2

aoi1_polygon = Polygon(
    [(77.55, 13.01), (77.58, 13.01), (77.58, 13.08), (77.55, 13.08), (77.55, 13.01)]
)

# get total bounds of all polygons above for STAC search and STAC index creation
bbox = aoi1_polygon.bounds



In [ ]:
# 1. Search for already created collections
print("1. Listing Available Collections")
print("--------------------------")
collections = Rasteret.list_collections(workspace_dir=workspace_dir)
for c in collections:
    print(f"Existing collection in workspace dir {workspace_dir}:")
    print(f"- {c['name']}: {c['data_source']}, {c['date_range']}, {c['size']} scenes")

In [ ]:
# You can load any existing collection in any folder
#processor = Rasteret.load_collection("bangalore_202401-01_landsat", workspace_dir=workspace_dir)


In [ ]:
print("2. Creating New Collection with custom name :", custom_name)
# Create new collection
processor = Rasteret(
    custom_name=custom_name,
    data_source=data_source,
    workspace_dir=workspace_dir,
    date_range=date_range
)
processor.create_collection(
    bbox=bbox,
    date_range=date_range,
    cloud_cover_lt=50,
)

In [ ]:
# With the processor object, you can now proceed to process the data with 
# as many geometries as you want and also choose the bands you want to process data for

print("\n3. Processing Data")
print("-----------------")
ds = processor.get_xarray(
    geometries=[aoi1_polygon],
    bands=["B04", "B08"],
    # give more STAC specific filters if requried
    cloud_cover_lt=20
)

# returns an xarray dataset with the data for the geometries and bands specified


In [ ]:
# Calculate NDVI
ndvi = (ds.B08 - ds.B04) / (ds.B08 + ds.B04)

# Create a new dataset with NDVI as a variable
ndvi_ds = xr.Dataset(
    {"NDVI": ndvi},
    coords=ds.coords,
    attrs=ds.attrs
)

In [ ]:
from rasteret.core.utils import save_per_geometry

# Save results in any directory
output_dir = Path(f"ndvi_results_{custom_name}")
output_dir.mkdir(exist_ok=True)

# Save the NDVI data for each geometry in a separate file
# provide file_prefix to name the files, and the xarray variable name to save
output_files = save_per_geometry(ndvi_ds, output_dir, file_prefix="ndvi", data_var="NDVI")

print("\nProcessed NDVI files:")
for geom_id, filepath in output_files.items():
    print(f"Geometry {geom_id}: {filepath}")
